# ETL PROCESS

Database (Snowflake DW)
This Ingest process uses the Snowflake data warehouse to hold the external data.
“Snowflake is an analytics data warehouse delivered as software as a service (SaaS). Snowflake’s data warehouse does not rely on an existing database or “big data” software platform like Hadoop. Snowflake’s data warehouse uses a new SQL database engine with a unique architecture designed for the cloud.”
For more information: https://www.snowflake.com/workloads/data-warehouse-modernization/

## Definition of functions
The necessary functions for the ETL process will be defined below.


## Warehouse Functions
Next we will define the functions that will allow us to interact between Python and our Warehouse
Connection for Snowflake DW
Note: Create venv file with variable user and password, this for security of our data.
This is a Personal warehouse, but the database and schemas have been isolated to work in this specific case.

In [2]:
import snowflake.connector
import os
from dotenv import load_dotenv
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas

"""
Set up connection
"""
load_dotenv()
con = snowflake.connector.connect(
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
    database='DEEL_ANALYTICS',
    schema='SRC_RAW_GLOBEPAY',

)



def put_data_in_snowflake( df, schema: str, table_name: str, ):
    df_1 = df.copy()
    return write_pandas(con, df_1, table_name=table_name.upper(), schema=schema.upper(), auto_create_table=True)

## Functions to handling file

In [3]:

def csv_to_df(filename):
    """
    Function for read csv File
    :param filename:
    :return: Dataframe parsing from CSV
    """
    df = pd.read_csv(filename, sep=',', encoding='UTF8')
    return df


def standardize_columns(df):
    """
    Function for standardize columns of a dataframe
    :param df:
    :return: Columns standardized
    """

    df_1 = df.copy()
    new_columns = []
    count = 1
    for x in df_1.columns:
        x.lower()
        mt = x.lower().maketrans("/'.,:¿?/()ÁÉÍÓÚáéíóú", '          aeiouaeiou')
        new_column = x.translate(mt).strip().replace(" ", "_").upper()
        new_columns.append(new_column)

    return new_columns


def get_columns_duplicated(df):
    """
    Function to get a list of columns duplicated in a dataframe
    :param df:
    :return: List of columns duplicated in a dataframe
    """
    duplicated_columns = []
    for c in df.columns:

        if df.columns.to_list().count(c) > 1:
            duplicated_columns.append(c)

    return list(set(duplicated_columns))


def rename_duplicated_columns(df):
    """
    Function for rename duplicated values in a dataframe
    :param df: Input dataframe
    :return: Dataframe with renamed columns
    """
    cols = []
    count = 1
    duplicated_columns = get_columns_duplicated(df)

    for duplicated_column in duplicated_columns:
        for column in df.columns:
            if column == duplicated_column:
                cols.append(f'{duplicated_column}_{count}')
                count += 1
                continue
            cols.append(column)
        df.columns = cols


## running pipeline

In [4]:
file_names = os.listdir('sources')
file_names

['Globepay Chargeback Report - Globepay Chargeback Report.csv',
 'Globepay Acceptance Report - Globepay Acceptance Report.csv']

In [6]:

for file in file_names:

    #1.  Read local File
    file_name = f"sources/{file}"
    df_raw = csv_to_df(file_name)

    #2. Handling column names
    df_raw.columns = standardize_columns(df_raw)

    #3. Check and handling duplicated columns
    rename_duplicated_columns(df_raw)
    table_name = file.replace('.csv','')

    #4. Send raw data to Snowflake :3
    put_data_in_snowflake(df_raw, 'SRC_RAW_GLOBEPAY',table_name )
    print("Everything was ok for table", table_name, " !")




Everything was ok for table Globepay Chargeback Report - Globepay Chargeback Report  !
Everything was ok for table Globepay Acceptance Report - Globepay Acceptance Report  !
